In [1]:
!nvidia-smi

Wed May  5 23:39:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Preparations

requirements

In [ ]:
!pip install transformers==4.4.0
!pip install pydub
!apt install ffmpeg
!pip install datasets
!pip install jiwer
!pip install python-Levenshtein

covert raw csv to compatible csv format

train/val split

In [8]:
import pandas as pd
df = pd.read_table("knnw_en_sub_edit.csv", sep = ";", header=0)
train_df = df.sample(frac=0.8, random_state=0)
train_df.to_csv("knnw_sub_train.csv", index=False)

val_df = df.drop(train_df.index)
val_df.to_csv("knnw_sub_val.csv", index=False)

prepare wav files

In [16]:
audio_path = "./knnw_en_mono.wav"
subtitle_lookup_path = "knnw_en_sub_edit.csv"
save_dir = "./wav_data/"

In [17]:
import os 
from pydub import AudioSegment


def preprocess_wav(audio_path, subtitle_lookup_path, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    audio = AudioSegment.from_file(audio_path, format="wav")
    total_duration = len(audio)
    subtitle_lookup = pd.read_table(subtitle_lookup_path, sep = ";", header=0)  # drop the first non-dialogue subtitle
    for i in range(len(subtitle_lookup)):
        start_time = subtitle_lookup.iloc[i, 1]
        stop_time = subtitle_lookup.iloc[i, 2]

        audio_item = audio[start_time: stop_time]
        audio_item.export(save_dir + str(subtitle_lookup.iloc[i, 0]) + ".wav", format="wav")

preprocess_wav(audio_path, subtitle_lookup_path, save_dir)

# Wav2Vec 2.0


In [1]:
import re
import json
import sys
import time
import random

import numpy as np
import pandas as pd

import torch

from pydub import AudioSegment
import librosa

import Levenshtein

from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2ForCTC

from datasets import load_dataset, load_metric
from datasets import ClassLabel
from IPython.display import display, HTML

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## preprocess data

load train/test data

In [ ]:
knnw = load_dataset('csv', data_files={'train': './knnw_sub_train.csv', 'test': './knnw_sub_val.csv'})


In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(knnw["train"])

data cleaning

In [4]:
def remove_chars(batch):
    batch['Text'] = batch['Text'].lower()
    null = 'null'
    batch['Text'] = re.sub(r'.*""', null, batch['Text'])
    batch['Text'] = batch['Text'].replace('?', '')
    batch['Text'] = batch['Text'].replace('!', '')
    batch['Text'] = batch['Text'].replace(',', '')
    batch['Text'] = batch['Text'].replace('-', ' ')
    batch['Text'] = batch['Text'].replace('"', '')
    batch['Text'] = batch['Text'].replace("“", '')
    batch['Text'] = batch['Text'].replace("”", '')
    batch['Text'] = batch['Text'].replace('...', '')
    batch['Text'] = batch['Text'].replace('é', 'e')
    batch['Text'] = batch['Text'].replace('21', 'twenty one')
    batch['Text'] = batch['Text'].replace('1200', 'twelve hundred')
    batch['Text'] = batch['Text'].replace('20th', 'twentieth')
    batch['Text'] = batch['Text'].replace('7:40', 'seven fourty')
    batch['Text'] = batch['Text'].replace('8:42', 'eight fourty two')
    batch['Text'] = batch['Text'].replace('1994', 'nineteen ninety four')
    batch['Text'] = batch['Text'].replace('9', 'nine')
    batch['Text'] = batch['Text'].replace('500', 'five hundred')
    batch['Text'] = re.sub(r'\(.*\)', '', batch['Text'])
    batch['Text'] = re.sub(r'[\w ]+: ', ' ', batch['Text'])
    batch['Text'] = re.sub(r' +', ' ', batch['Text'])
    if batch['Text'][0] == ' ':
        batch['Text'] = batch['Text'][1:]
    batch['Text'] = re.sub(r'\[.*\] *', ' ', batch['Text'])
    if batch['Text'] == '':
        batch['Text'] = null
    
    # changes from our hw4p2 remove_chars():
    batch['Text'] = '<s>' + batch['Text'].upper() + '</s>'
    batch['Text'] = batch['Text'].replace(" ", "|")
    batch['Text'] = batch['Text'].replace(".", "")
    
    return batch


In [ ]:
knnw = knnw.map(remove_chars)


In [ ]:
show_random_elements(knnw['train'])


load wav files into arrays

In [ ]:
import pydub
wav_dir = './wav_data/'

def speech_file_to_array_fn(batch):
    speech_array, sr = librosa.load(wav_dir + str(batch['Number']) + ".wav", sr=16000)
    batch["speech"] = speech_array
    batch["sampling_rate"] = sr
    batch["target_text"] = batch["Text"]
    return batch

knnw = knnw.map(speech_file_to_array_fn, num_proc=4)

check examples

In [9]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(knnw["train"]))
print(knnw["train"][rand_int]["target_text"])
ipd.Audio(data=np.asarray(knnw["train"][rand_int]["speech"]), autoplay=True, rate=16000)

rand_int = random.randint(0, len(knnw["test"]))
print(knnw["test"][rand_int]["target_text"])
ipd.Audio(data=np.asarray(knnw["test"][rand_int]["speech"]), autoplay=True, rate=16000)

<s>I|CAN'T|THERE|ARE|LIQUOR|LAWS</s>
<s>HEY|YOU|UH|YES</s>


In [10]:
rand_int = random.randint(0, len(knnw["train"]))

print("Target text:", knnw["train"][rand_int]["target_text"])
print("Input array shape:", np.asarray(knnw["train"][rand_int]["speech"]).shape)
print("Sampling rate:", knnw["train"][rand_int]["sampling_rate"])

Target text: <s>HOLD|ON|THIS|WON'T|TAKE|LONG</s>
Input array shape: (32320,)
Sampling rate: 16000


In [11]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")


convert chars into ids

In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    # print(batch['target_text'])
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    # print(batch["labels"])
    return batch

knnw_prepared = knnw.map(prepare_dataset, batch_size=8, num_proc=4, batched=True)
# knnw_prepared = knnw_preapared.map(prepare_dataset, remove_columns=knnw_prepared.column_names["test"], batch_size=8, num_proc=4, batched=True)


## setup trainer

In [14]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [15]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


In [16]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    pred_str = [s.replace("<s>", "").replace("</s>", "") for s in pred_str]
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    label_str = [s.replace("<s>", "").replace("</s>", "") for s in label_str]

    distances = np.vectorize(lambda x,y: Levenshtein.distance(x,y))(pred_str, label_str)
    print(pred_str)
    print(label_str)

    distance = np.mean(distances)

    return {"edit_distance": distance}

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base-960h",
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)


In [47]:
model.freeze_feature_extractor()

In [48]:
model = model.to(device)

Eval pretrained wav2vec without finetuning

In [ ]:
model.eval()
def map_to_result(batch):
  model.to("cuda")
  input_values = processor(
      batch["speech"], 
      sampling_rate=batch["sampling_rate"], 
      return_tensors="pt"
  ).input_values.to(device)

  with torch.no_grad():
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["pred_str"] = batch["pred_str"].replace("<s>", "").replace("</s>", "").lower()

  batch['target_text'] = batch['target_text'].replace("|", " ").replace("<s>", "").replace("</s>", "").lower()

  return batch


results = knnw["test"].map(map_to_result)

# compute Levenshtein distance

distances = np.vectorize(lambda x,y: Levenshtein.distance(x,y))(results['target_text'], results["pred_str"])
avg_distance = sum(distances) / len(results['target_text'])

print("pretrained Wav2vec 2.0 performance: ", avg_distance)

In [52]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./wav2vec2-base-knnw-demo",
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="epoch",
  num_train_epochs=20,
  fp16=True,
  learning_rate=5e-5,
  weight_decay=5e-7,
  warmup_steps=10,
  save_total_limit=2,
)

In [53]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=knnw_prepared["train"],
    eval_dataset=knnw_prepared["test"],
    tokenizer=processor.feature_extractor,
)

In [54]:
model.train()
trainer.train()

Epoch,Training Loss,Validation Loss,Edit Distance,Runtime,Samples Per Second
1,No log,2.053681,9.428016,15.910500,16.153000
2,No log,1.502164,8.992218,15.825900,16.239000
3,No log,1.611551,9.525292,15.929900,16.133000
4,No log,1.625351,9.256809,16.002000,16.061000
5,No log,1.530053,8.750973,15.940900,16.122000
6,No log,1.318431,8.081712,16.133900,15.929000
7,No log,1.298909,7.844358,16.085700,15.977000
8,No log,1.292134,7.575875,16.188600,15.875000
9,No log,1.306978,7.638132,16.211800,15.853000
10,No log,1.392978,7.513619,16.752100,15.341000


['TE DA STARFEL', "WHAT'S  ITSO KETHERN A COW LIKE AR", "H'M TOL THAT SOME PON OF EVERY WISH WILL BE HEARD", 'BE LADY AYLASIDE OF THE TRUTH IN THOSE WORDS', 'AH  THE VERI LOOKAT DACTOR RAIN WILL START', 'ONE DOUN REACH TAN MI SHA O EXTOOT O BRAS A DOON', 'WITH A BODNOLBI DANT THAT IS CATERAND TA O E CURISESON', 'IS MAKE A SACK FOR WON WE SAY NIS TO NEAT DOUGAN', 'TULKY', 'YNANIS', 'RAZY BUT YOURSELF FOR LOV', 'BYO LET HIM BE HELLOW IT O YE NEXT MONTH', 'A COMET THAT COMES ONLY ONCE EVERY TWELVE HUNDRED YEARS', 'JUST MAKE UP WITH HIM ALREADY ELC', "IT'S NOT DONT MON ACCORDING TO JACKSO SPOKE'S PERSON THEY'RE PREPARING FOR THE CELESTIAL SHOW OF THE MILLENNIUM", 'WITALI SAIL YOU ACTUALLY BOTHERED TO NEW YOUR HAIR TO DAY', 'TEK ARSISE SOMETHING TORTALLY POSSESSED YOUR BODY', 'WOULD JID JUST GIVE IT A RESENT WITH ALL THE SICULT MANTAS', 'AND TOLD ME WHON ALL THE THINK ARE ACHIEF THAT WE CAN CREATE US SAFE AND SECURE COMMUNITY', "A HARRY'S EN WI WORN OUT  BETTAK JOSS BETWEEN NOW", 'FEMINA LA

TrainOutput(global_step=660, training_loss=1.1951010732939749, metrics={'train_runtime': 1669.7553, 'train_samples_per_second': 0.395, 'total_flos': 6.236834036023296e+17, 'epoch': 20.0, 'init_mem_cpu_alloc_delta': 1896074, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 801974817, 'init_mem_gpu_peaked_delta': 1518464512, 'train_mem_cpu_alloc_delta': 689345, 'train_mem_gpu_alloc_delta': 1097235968, 'train_mem_cpu_peaked_delta': 129949052, 'train_mem_gpu_peaked_delta': 4216145408})

# Evaluate

In [ ]:
## uncomment the following lines if you want to test the raw pretrained model
## otherwise it tests the above finetuned model

# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
# model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

In [ ]:
def map_to_result(batch):
  model.eval()
  model.to("cuda")
  input_values = processor(
      batch["speech"], 
      sampling_rate=batch["sampling_rate"], 
      return_tensors="pt"
  ).input_values.to(device)

  with torch.no_grad():
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["pred_str"] = batch["pred_str"].replace("<s>", "").replace("</s>", "").lower()

  batch['target_text'] = batch['target_text'].replace("|", " ").replace("<s>", "").replace("</s>", "").lower()

  return batch


pred = knnw["test"].map(map_to_result)

In [69]:
pred_df = pd.DataFrame({"prediction": pred["pred_str"], "target": pred['target_text']})

In [70]:
pred_df.head()

,prediction,target
0,the day a starfell,the day a star fell
1,we'll see it togetherand cow like tru,let's say it together on the count of three oh
2,am so that some part of every wish will be heard,ah i'm told that some part of every wish will ...
3,but lady ayla side of the truth in those words,but lately i've lost sight of the truth in tho...
4,ah the very moment that the rain well stopped,ah the very moment that the rain will stop
